In [6]:
import pandas as pd
import csv
import sys
import os


In [7]:
# --- FIX 1: Increase field size limit ---
csv.field_size_limit(sys.maxsize)

131072

# ==================================================
# Step 1: Load Communication Data
# ==================================================

In [8]:
# --- Chunked CSV loading function (handles big Enron files safely) ---
def load_comm_data_chunked(path, chunksize=10000, max_rows=200000):
    """
    Loads large Enron communication CSVs safely by chunking and concatenating
    only a preview sample for analysis setup.
    """
    chunks = []
    rows_read = 0
    for chunk in pd.read_csv(path, chunksize=chunksize, on_bad_lines='skip', encoding='utf-8', engine='python'):
        chunks.append(chunk)
        rows_read += len(chunk)
        if rows_read >= max_rows:  # limit for memory safety
            break
    df = pd.concat(chunks, ignore_index=True)
    return df

# ==================================================
# Step 2: Load or Generate System Logs
# ==================================================

In [9]:
# --- System Logs Loader (if already have system_logs.csv) ---
def load_or_generate_system_logs(path="/content/system_logs.csv"):
    if os.path.exists(path):
        print(f"📂 Found existing system_logs.csv at {path}")
        return pd.read_csv(path)
    else:
        print("⚙️ Generating synthetic system_logs.csv for UBA correlation...")

        import numpy as np
        import datetime

        users = [f"user{i}@enron.com" for i in range(1, 21)]
        activities = ["login", "logout", "file_access", "email_sent", "download", "upload", "config_change"]

        data = []
        for _ in range(1000):
            data.append({
                "user": np.random.choice(users),
                "activity": np.random.choice(activities),
                "timestamp": datetime.datetime.now() - datetime.timedelta(minutes=np.random.randint(0, 10000)),
                "status": np.random.choice(["success", "fail"]),
                "source_ip": f"10.0.{np.random.randint(0, 255)}.{np.random.randint(0, 255)}"
            })

        df = pd.DataFrame(data)
        df.to_csv(path, index=False)
        print(f"✅ Synthetic system logs saved to {path}")
        return df



# ==================================================
# Load datasets
# ==================================================

In [10]:
comm_path = "/content/enron_recleaned.csv"
sys_path = "/content/system_logs.csv"

comm_df = load_comm_data_chunked(comm_path)
sys_df = load_or_generate_system_logs(sys_path)

print("✅ Communication data loaded:", comm_df.shape)
print("✅ System logs loaded:", sys_df.shape)

print("\n📊 Communication Data Sample:")
display(comm_df.head())

print("\n📊 System Logs Sample:")
display(sys_df.head())

📂 Found existing system_logs.csv at /content/system_logs.csv
✅ Communication data loaded: (57895, 9)
✅ System logs loaded: (713583, 8)

📊 Communication Data Sample:


,file,message,from,to,cc,bcc,date,subject,cleaned_message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,phillip.allen@enron.com,tim.belden@enron.com,NaN,NaN,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",Mime-Version: 1.0,mimevers contenttyp textplain charsetusascii c...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,phillip.allen@enron.com,john.lavorato@enron.com,NaN,NaN,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",Re:,mimevers contenttyp textplain charsetusascii c...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,phillip.allen@enron.com,leah.arsdall@enron.com,NaN,NaN,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",Re: test,mimevers contenttyp textplain charsetusascii c...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,phillip.allen@enron.com,randall.gay@enron.com,NaN,NaN,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",Mime-Version: 1.0,mimevers contenttyp textplain charsetusascii c...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,phillip.allen@enron.com,greg.piper@enron.com,NaN,NaN,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",Re: Hello,mimevers contenttyp textplain charsetusascii c...



📊 System Logs Sample:


,timestamp,user,event_type,resource,action_status,device,location,severity_score
0,2001-05-20,phillip.allen@enron.com,file_access,usb_drive,failure,mobile,London,0.46
1,2002-01-01,phillip.allen@enron.com,login_success,hr.db,success,mobile,Houston,0.95
2,2001-02-03,phillip.allen@enron.com,file_access,hr.db,failure,desktop,Tokyo,0.66
3,2001-09-16,phillip.allen@enron.com,usb_insert,contracts.pdf,failure,vpn,Tokyo,0.23
4,2000-06-25,phillip.allen@enron.com,usb_insert,server01,failure,desktop,London,0.36


# ===================================
# Step 3: Compute UBA Risk Score
# ===================================

In [13]:
# --- Load System Logs ---
def load_system_logs(path):
    sys_df = pd.read_csv(path)
    sys_df['timestamp'] = pd.to_datetime(sys_df['timestamp'], errors='coerce')
    return sys_df

# --- Compute User-Level Behavioral Features ---
def compute_user_behavior(sys_df):
    behavior_df = sys_df.groupby("user").agg(
        failed_logins=("action_status", lambda x: (x == "failure").sum()),
        total_logins=("event_type", lambda x: (x == "login_success").sum()),
        usb_usage=("event_type", lambda x: (x == "usb_insert").sum()),
        file_failures=("event_type", lambda x: ((x == "file_access") & (sys_df.loc[x.index, "action_status"] == "failure")).sum()),
        unique_locations=("location", "nunique"),
        avg_severity=("severity_score", "mean")
    ).reset_index()

    return behavior_df

# --- Compute Risk Score ---
def compute_risk_score(df):
    df["risk_score"] = (
        0.25 * (df["failed_logins"] / (df["failed_logins"].max() + 1)) +
        0.25 * (df["usb_usage"] / (df["usb_usage"].max() + 1)) +
        0.2 * (df["file_failures"] / (df["file_failures"].max() + 1)) +
        0.15 * (df["unique_locations"] / (df["unique_locations"].max() + 1)) +
        0.15 * df["avg_severity"]
    )
    df["risk_score"] = df["risk_score"].clip(0, 1)
    return df

# --- Example Usage ---
#sys_path = "/content/system_logs.csv"
#sys_df = load_system_logs(sys_path)

uba_df = compute_user_behavior(sys_df)
uba_df = compute_risk_score(uba_df)

print("✅ User Behavior Analytics Summary:")
display(uba_df.head())

✅ User Behavior Analytics Summary:


,user,failed_logins,total_logins,usb_usage,file_failures,unique_locations,avg_severity,risk_score
0,'todd'.delahoussaye@enron.com,14,2,1,2,5,0.557857,0.356970
1,--migrated--bmishkin@ercot.com,25,4,6,3,5,0.512857,0.518320
2,--migrated--dodle@ercot.com,15,4,3,2,5,0.606061,0.400557
3,-nikole@excite.com,14,1,4,1,5,0.603462,0.389747
4,-persson@ricemail.ricefinancial.com,13,3,3,2,5,0.554583,0.378947


# ============================================
# Step 4: Communication-Based Risk Aggregation
# =============================================

In [16]:
def compute_comm_risk(comm_df):
    """Compute sender-level communication risk using behavioral and linguistic indicators."""

    # Replace missing text fields to avoid aggregation errors
    comm_df["cleaned_message"] = comm_df["cleaned_message"].fillna("")
    comm_df["subject"] = comm_df["subject"].fillna("")
    comm_df["cc"] = comm_df["cc"].fillna("")

    # Compute features per sender
    comm_features = comm_df.groupby("from").agg(
        messages_sent=("cleaned_message", "count"),
        unique_receivers=("to", pd.Series.nunique),
        avg_message_length=("cleaned_message", lambda x: np.mean(x.str.len())),
        cc_count=("cc", lambda x: x.str.len().gt(0).sum()),
        subject_activity=("subject", lambda x: x.str.len().gt(0).sum())
    ).reset_index()

    # Normalize and compute communication risk score
    comm_features["comm_risk"] = (
        0.3 * (comm_features["messages_sent"] / (comm_features["messages_sent"].max() + 1)) +
        0.25 * (comm_features["unique_receivers"] / (comm_features["unique_receivers"].max() + 1)) +
        0.25 * (comm_features["avg_message_length"] / (comm_features["avg_message_length"].max() + 1)) +
        0.1 * (comm_features["cc_count"] / (comm_features["cc_count"].max() + 1)) +
        0.1 * (comm_features["subject_activity"] / (comm_features["subject_activity"].max() + 1))
    )
    comm_features["comm_risk"] = np.clip(comm_features["comm_risk"], 0, 1)

    # Rename for merging consistency
    comm_features = comm_features.rename(columns={"from": "user"})

    return comm_features

# Example usage
comm_features = compute_comm_risk(comm_df)
print("✅ Communication risk features computed:", comm_features.shape)
display(comm_features.head())

✅ Communication risk features computed: (4023, 7)


,user,messages_sent,unique_receivers,avg_message_length,cc_count,subject_activity,comm_risk
0,'todd'.delahoussaye@enron.com,1,1,261.0,1,1,0.001642
1,0_19619_e719a090-7eca-d011-9d39-0000f84121eb_u...,1,1,1504.0,0,1,0.006128
2,0_19653_e719a090-7eca-d011-9d39-0000f84121eb_u...,1,1,2231.0,0,1,0.008818
3,0_19812_e719a090-7eca-d011-9d39-0000f84121eb_u...,1,1,8505.0,0,1,0.032033
4,1.10969419.-18@multexinvestornetwork.com,2,1,712.5,0,2,0.003277


# =================================
# Step 5: Merge & Correlate Risks
# =================================


In [17]:
def correlate_risks(uba_df, comm_features):
    """
    Merge user behavior analytics (UBA) risk scores with communication-based risk scores,
    then compute a final combined risk ranking.
    """
    # Merge on consistent user column
    combined = uba_df.merge(comm_features, on="user", how="outer").fillna(0)

    # Compute weighted final risk score (adjust weights as desired)
    combined["final_risk"] = 0.6 * combined["risk_score"] + 0.4 * combined["comm_risk"]

    # Normalize for readability (0–1 scale)
    combined["final_risk"] = combined["final_risk"] / combined["final_risk"].max()

    # Sort descending by total risk
    combined = combined.sort_values("final_risk", ascending=False)

    print("✅ Correlation complete:", combined.shape)
    print("\n Top 5 Potential Insider Threats:")
    print(combined[["user", "risk_score", "comm_risk", "final_risk"]].head())

    return combined


# Run correlation
correlated_df = correlate_risks(uba_df, comm_features)

✅ Correlation complete: (20328, 15)

 Top 5 Potential Insider Threats:
                                user  risk_score  comm_risk  final_risk
9475           john.arnold@enron.com    0.640869   0.449959    1.000000
6044             eric.bass@enron.com    0.352394   0.621539    0.814966
16841           sally.beck@enron.com    0.286250   0.684175    0.789046
3388   christa@crossborderenergy.com    0.714908   0.000000    0.759860
18817      themarshalls1@hotmail.com    0.601813   0.147337    0.744055


# ===============================
# Step 6: Save / Display Results
# ================================

In [18]:
# Select top risky users (adjust count if needed)
top_users = correlated_df[["user", "risk_score", "comm_risk", "final_risk"]].head(20)

print("\n🔍 Top 20 Risky Users:")
display(top_users.style.background_gradient(subset=["final_risk"], cmap="Reds"))

# Save results to CSV
output_path = "/content/final_correlated_risk.csv"
correlated_df.to_csv(output_path, index=False)

print(f"📁 Saved final correlated risk results to: {output_path}")


🔍 Top 20 Risky Users:


,user,risk_score,comm_risk,final_risk
9475,john.arnold@enron.com,0.640869,0.449959,1.000000
6044,eric.bass@enron.com,0.352394,0.621539,0.814966
16841,sally.beck@enron.com,0.286250,0.684175,0.789046
3388,christa@crossborderenergy.com,0.714908,0.000000,0.759860
18817,themarshalls1@hotmail.com,0.601813,0.147337,0.744055
9715,jordan.w.cowman@bakernet.com,0.538713,0.230714,0.736067
16488,roger.yang@enron.com,0.674148,0.016997,0.728581
13552,mortl98@freemail.hu,0.681505,0.000000,0.724357
15071,phillip.allen@enron.com,0.494970,0.277767,0.722914
6664,freetexas@msn.com,0.679514,0.000000,0.722241


📁 Saved final correlated risk results to: /content/final_correlated_risk.csv
